# Cubeviz Aperture Photometry

Concept notebook to demonstrate aperture photometry in Cubeviz.

In [ ]:
import numpy as np
from astropy import units as u
from astropy.io import fits
from astropy.nddata import StdDevUncertainty
from astropy.wcs import WCS
from regions import RectanglePixelRegion, PixCoord
from specutils import Spectrum1D

from jdaviz import Cubeviz

## Cube with linear flux unit

In [ ]:
# Same data as the one in conftest.py

def image_cube_hdu_obj_microns():
    # Basic rectangle ramp for aperture photometry test.
    a = np.zeros((8, 9, 10)).astype(np.float32)  # (nz, ny, nx)
    for i in range(8):
        a[i, :5, :3] = i + 1
    flux_hdu = fits.ImageHDU(a)
    flux_hdu.name = 'FLUX'

    uncert_hdu = fits.ImageHDU(np.zeros((8, 9, 10)).astype(np.float32))
    uncert_hdu.name = 'ERR'

    mask_hdu = fits.ImageHDU(np.ones((8, 9, 10)).astype(np.uint16))
    mask_hdu.name = 'MASK'

    wcs = {
        'WCSAXES': 3, 'CRPIX1': 38.0, 'CRPIX2': 38.0, 'CRPIX3': 1.0,
        'CRVAL1': 205.4384, 'CRVAL2': 27.004754, 'CRVAL3': 4.890499866509344,
        'CTYPE1': 'RA---TAN', 'CTYPE2': 'DEC--TAN', 'CTYPE3': 'WAVE',
        'CUNIT1': 'deg', 'CUNIT2': 'deg', 'CUNIT3': 'um',
        'CDELT1': 3.61111097865634E-05, 'CDELT2': 3.61111097865634E-05, 'CDELT3': 0.001000000047497451,  # noqa
        'PC1_1 ': -1.0, 'PC1_2 ': 0.0, 'PC1_3 ': 0,
        'PC2_1 ': 0.0, 'PC2_2 ': 1.0, 'PC2_3 ': 0,
        'PC3_1 ': 0, 'PC3_2 ': 0, 'PC3_3 ': 1,
        'DISPAXIS': 2, 'VELOSYS': -2538.02,
        'SPECSYS': 'BARYCENT', 'RADESYS': 'ICRS', 'EQUINOX': 2000.0,
        'LONPOLE': 180.0, 'LATPOLE': 27.004754,
        'MJDREFI': 0.0, 'MJDREFF': 0.0, 'DATE-OBS': '2014-03-30'}

    flux_hdu.header.update(wcs)
    flux_hdu.header['BUNIT'] = '1E-17 erg*s^-1*cm^-2*Angstrom^-1'

    uncert_hdu.header['BUNIT'] = '1E-17 erg*s^-1*cm^-2*Angstrom^-1'

    return fits.HDUList([fits.PrimaryHDU(), flux_hdu, uncert_hdu, mask_hdu])

test_cube = image_cube_hdu_obj_microns()

In [ ]:
cubeviz = Cubeviz()

In [ ]:
cubeviz.load_data(test_cube, data_label="test")

In [ ]:
cubeviz.show()

In [ ]:
cubeviz.default_viewer._obj.state.reset_limits()

In [ ]:
aper = RectanglePixelRegion(center=PixCoord(x=1, y=2), width=3, height=5)

In [ ]:
cubeviz.load_regions(aper)

In [ ]:
plg = cubeviz.plugins["Aperture Photometry"]._obj
cube_slice_plg = cubeviz.plugins["Slice"]._obj

In [ ]:
plg.open_in_tray()

In [ ]:
plg.dataset_selected = "test[FLUX]"

In [ ]:
plg.aperture_selected = "Subset 1"

In [ ]:
plg.vue_do_aper_phot()

### Change slice

In [ ]:
cube_slice_plg.slice = 0

In [ ]:
plg.vue_do_aper_phot()

In [ ]:
# Change back
cube_slice_plg.slice = 4

### Collapse

In [ ]:
collapse_plg = cubeviz.plugins["Collapse"]._obj

In [ ]:
collapse_plg.vue_collapse()

In [ ]:
cubeviz.app.add_data_to_viewer("uncert-viewer", "test[FLUX] collapsed")

In [ ]:
plg.dataset_selected = "test[FLUX] collapsed"

In [ ]:
plg.vue_do_aper_phot()

### Moment map

In [ ]:
moment_plg = cubeviz.plugins["Moment Maps"]

In [ ]:
m = moment_plg.calculate_moment()

In [ ]:
cubeviz.app.add_data_to_viewer("uncert-viewer", "test[FLUX] moment 0")

In [ ]:
plg.dataset_selected = "test[FLUX] moment 0"

In [ ]:
plg.vue_do_aper_phot()

### Gaussian Smooth

In [ ]:
gauss_plg = cubeviz.plugins["Gaussian Smooth"]._obj

In [ ]:
gauss_plg.mode_selected = "Spatial"

In [ ]:
g = gauss_plg.smooth()

In [ ]:
cubeviz.app.add_data_to_viewer("uncert-viewer", "test[FLUX] spatial-smooth stddev-1.0")

In [ ]:
plg.dataset_selected = "test[FLUX] spatial-smooth stddev-1.0"

In [ ]:
plg.vue_do_aper_phot()

### Show results

In [ ]:
t = cubeviz.get_aperture_photometry_results()
t  # Should have 5 rows

## Cube with MJy/sr

In [ ]:
def _create_spectrum1d_cube_with_fluxunit(fluxunit=u.Jy, shape=(2, 2, 4), with_uncerts=False):
    # nz=2 nx=2 ny=4
    flux = np.arange(np.prod(shape)).reshape(shape) * fluxunit
    wcs_dict = {"CTYPE1": "RA---TAN", "CTYPE2": "DEC--TAN", "CTYPE3": "WAVE-LOG",
                "CRVAL1": 205, "CRVAL2": 27, "CRVAL3": 4.622e-7,
                "CDELT1": -0.0001, "CDELT2": 0.0001, "CDELT3": 8e-11,
                "CRPIX1": 0, "CRPIX2": 0, "CRPIX3": 0,
                # Need these for aperture photometry test.
                 "TELESCOP": "JWST", "BUNIT": fluxunit.to_string(), "PIXAR_A2": 0.01}
    w = WCS(wcs_dict)
    if with_uncerts:
        uncert = StdDevUncertainty(np.abs(np.random.normal(flux) * u.Jy))

        return Spectrum1D(flux=flux,
                          uncertainty=uncert,
                          wcs=w,
                          meta=wcs_dict)
    else:
        return Spectrum1D(flux=flux, wcs=w, meta=wcs_dict)

test_cube2 = _create_spectrum1d_cube_with_fluxunit(fluxunit=u.MJy/u.sr)

In [ ]:
cubeviz2 = Cubeviz()

In [ ]:
cubeviz2.load_data(test_cube2, data_label="test")

In [ ]:
cubeviz2.show()

In [ ]:
cubeviz2.default_viewer._obj.state.reset_limits()

In [ ]:
aper2 = RectanglePixelRegion(center=PixCoord(x=1, y=3), width=1, height=1)
bg2 = RectanglePixelRegion(center=PixCoord(x=0, y=2), width=1, height=1)
cubeviz2.load_regions([aper2, bg2])

In [ ]:
plg2 = cubeviz2.plugins["Aperture Photometry"]._obj

In [ ]:
plg2.open_in_tray()

In [ ]:
plg2.dataset_selected = "test[FLUX]"

In [ ]:
plg2.aperture_selected = "Subset 1"

In [ ]:
plg2.background_selected = "Subset 2"

In [ ]:
plg2.vue_do_aper_phot()

In [ ]:
t2 = cubeviz2.get_aperture_photometry_results()
t2